In [1]:
%run algebra.py
%run cache.py
%run costs.py
%run features.py
%run gradients.py
%run helpers.py
%run model.py
%run models.py
%run splits.py

import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import csv
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
SUB_SAMPLE = True
CACHE_DIR = "test/cache/" if SUB_SAMPLE else "cache/"
SUBMISSIONS_DIR = "test/submissions/" if SUB_SAMPLE else "submissions/"

In [3]:
def load_csv_data(data_path, sub_sample=True):
    """Loads data and returns y (class labels), tX (features) and ids (event ids)"""
    y = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    x = np.genfromtxt(data_path, delimiter=",", skip_header=1)
    ids = x[:, 0].astype(np.int)
    input_data = x[:, 2:]

    # convert class labels from strings to binary (-1,1)
    yb = np.ones(len(y))
    yb[np.where(y=='b')] = -1

    # sub-sample
    if sub_sample:
        yb = yb[::50]
        input_data = input_data[::50]
        ids = ids[::50]

    return yb, input_data, ids

In [4]:
y, x, ids = load_csv_data('data/train.csv', SUB_SAMPLE)

### Logistic Regression

In [15]:
class First_Order_Logistic_Regression_Model2(Model):

    def prepare(self, x, y, h):
        
        degree = int(h['degree'])

        x = remove_errors(x)
        x = remove_outliers(x)
        x = standardize_all(x)
        x = remove_nan_features(x)
        x = build_poly(x, degree)
        
        return x, y

    def fit(self, x, y, h={}):
        
        batch_size = int(h['batch_size'])
        n_iters = int(h['n_iters'])
        gamma = float(h['gamma'])
        
        initial_w = np.zeros(x.shape[1])
        return logistic_regression(y, x, initial_w, batch_size, n_iters, gamma)
    
    def test(self, x, y, w, h):
                
        return { 'mse': compute_mse(y, x, w) }

In [18]:
myModel = CrossValidationModel(First_Order_Logistic_Regression_Model2())

n_iters = [10]
batch_size = [1]
degrees = np.arange(12)
gammas = np.logspace(-10, -5, 10)

hs={
    'n_iters': n_iters,
    'batch_size': batch_size,
    'degree': degrees,
    'gamma': gammas,
    'k_fold': [4],
    'seed': [0]
}

res = myModel.evaluate(x, y, hs, filename=CACHE_DIR+'Logistic_Regression_TESTTTTT')

plot_heatmap(res, hs, 'avg_mse_te', 'degree', 'gamma')

res_mse = np.vectorize(lambda x: x['mse'])(res)

#plot_heatmap(res, hs, 'mse', 'degree', 'gamma')
find_arg_min(res, 'mse')

IndexError: invalid index to scalar variable.

### Second Order Logistic Regression